In [1]:
# Autoreload extension for Jupyter notebooks
%load_ext autoreload
%autoreload 2

# Manual reload
from importlib import reload

# Magic commands for inline plotting and high-res figures
%matplotlib inline
%config InlineBackend.figure_format ='retina'

# Standard library imports
import os
import sys
import pickle
from pprint import pprint

# Third-party library imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
from tqdm import tqdm

# Add the parent directory to sys.path (if needed)
# from pathlib import Path
# sys.path.append(str(Path(os.getcwd()).parent))

# Other modules (if needed)

# Pandas and plotting configurations
pd.set_option('display.max_rows', 1000)
idx = pd.IndexSlice

# Plotting configurations
DFT_FIG_WIDTH_WIDE = 11
DFT_FIG_WIDTH = 5.3
COLORS = plt.get_cmap('tab10')
# Choose in ['Pastel1', 'Pastel2', 'Paired', 'Accent', 'Dark2', 'Set1', 'Set2', 'Set3', 
# 'tab10', 'tab20', 'tab20b', 'tab20c']

jtplot.style(context="paper", theme="grade3", grid=True, ticks=True, fscale=1.1)
plt.rcParams.update(
    {
        "grid.alpha": 0.5,
        "axes.prop_cycle": plt.cycler("color", plt.cm.tab10.colors),
    }
)
CMU = "CMU Sans Serif"  # Default font in LaTeX

# Utility functions
def write_pickle(path, obj):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

def read_pickle(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [69]:
from ldpc_post_selection.bplsd_decoder import SoftOutputsBpLsdDecoder
from simulations.utils.build_circuit import build_BB_circuit

circuit = build_BB_circuit(n=144, T=12, p=3e-3)
decoder = SoftOutputsBpLsdDecoder(circuit=circuit)

In [74]:
fails = []
for seed in range(1):
    fail, soft_outputs = decoder.simulate_single(
        sliding_window=True, window_size=9, commit_size=1, verbose=False, seed=seed
    )
    fails.append(fail)
sum(fails) / len(fails)

ValueError: Decoding outcome invalid

In [73]:
fails = []
for seed in range(100):
    fail, soft_outputs = decoder.simulate_single(
        sliding_window=False, verbose=False, seed=seed
    )
    fails.append(fail)
sum(fails) / len(fails)

0.01